# **Introduction to Pandas III**

Pandas berisi struktur data dan alat manipulasi data yang dirancang untuk pembersihan data dan analisis yang cepat dan mudah dalam Python. Pandas sering digunakan bersama dengan pustaka komputasi numerik seperti NumPy dan SciPy, pustaka analitik seperti statsmodels dan scikit-learn, dan pustaka visualisasi data seperti matplotlib. Pandas mengadopsi secara signifikan bagian dari gaya idiomatis komputasi berbasis larik NumPy, terutama berbasis larik dan preferensi untuk pemrosesan data tanpa perulangan.

Sejak menjadi open source pada tahun 2010, pandas telah berkembang menjadi cukup besar yang dapat diterapkan dalam berbagai kasus penggunaan di dunia nyata. Pengembang komunitas telah berkembang menjadi lebih dari 800 kontributor yang berbeda, yang telah membantu membangun proyek ini karena mereka telah menggunakannya untuk memecahkan masalah data sehari-hari.

___

## **1. Aggregations and Grouping using groupby()**

Agregasi merupakan proses pengelompokkan dan penyusunan data menjadi sebuah dataframe. Untuk tujuan ini, kita dapat menjalankan `groupby()`.

In [1]:
import pandas as pd 
import numpy as np 
pd.options.display.max_columns=20

Pertama-tama, kita baca terlebih dahulu file `unicorn_companies_clean.csv`.

In [2]:
df = pd.read_csv('unicorn_companies_clean.csv')
df.head()

,Company,Valuation,Date Joined,Industry,City,Country,Continent,Year Founded,Select Investors,Year Joined,Years_To_Unicorn,Valuation Number (in B),Valuation Class,Number Investors
0,Bytedance,$180B,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,"Sequoia Capital China, SIG Asia Investments, S...",2017,5,180,High,4
1,SpaceX,$100B,2012-12-01,Other,Hawthorne,United States,North America,2002,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2012,10,100,High,3
2,SHEIN,$100B,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,"Tiger Global Management, Sequoia Capital China...",2018,10,100,High,3
3,Stripe,$95B,2014-01-23,Fintech,San Francisco,United States,North America,2010,"Khosla Ventures, LowercaseCapital, capitalG",2014,4,95,High,3
4,Klarna,$46B,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,"Institutional Venture Partners, Sequoia Capita...",2011,6,46,High,3


Misal kita ingin menampilkan total valuasi berdasarkan `Continent`. Kita dapat menggunakan `groupby()` yang dipadankan dengan fungsi agregat `sum()`.

In [7]:
# cara penulisan 1
df.groupby('Continent').sum(numeric_only=True)[['Valuation Number (in B)']]

,Valuation Number (in B)
Continent,
Africa,5
Asia,1041
Europe,503
North America,1985
Oceania,56
South America,48


In [10]:
# cara penulisan 2
df.groupby('Continent')[['Valuation Number (in B)']].sum()

,Valuation Number (in B)
Continent,
Africa,5
Asia,1041
Europe,503
North America,1985
Oceania,56
South America,48


Agar kolom `Continent` tidak menjadi indeks melainkan sebagai kolom, kita bisa menggunakan `reset_index()` setelah fungsi agregat atau menggunakan parameter `as_index=False` dalam fungsi `groupby()`

In [12]:
# reset index
df.groupby('Continent')[['Valuation Number (in B)']].sum().reset_index()

,Continent,Valuation Number (in B)
0,Africa,5
1,Asia,1041
2,Europe,503
3,North America,1985
4,Oceania,56
5,South America,48


In [14]:
# parameter as_index
df.groupby('Continent', as_index=False)[['Valuation Number (in B)']].sum()

,Continent,Valuation Number (in B)
0,Africa,5
1,Asia,1041
2,Europe,503
3,North America,1985
4,Oceania,56
5,South America,48


Kita juga dapat mengelompokkan berdasarkan lebih dari satu kolom dengan cara memasukkannya ke dalam list. Misal kita ingin menghitung rata-rata jumlah investor dikelompokkan berdasarkan `Continent` dan `Valuation Class`.

In [19]:
df['Continent'].unique()

array(['Asia', 'North America', 'Europe', 'Oceania', 'South America',
       'Africa'], dtype=object)

In [26]:
df.groupby(['Continent', 'Valuation Class'], as_index=False)[['Number Investors']].mean()

,Continent,Valuation Class,Number Investors
0,Africa,Low,2.666667
1,Asia,High,2.825000
2,Asia,Low,2.809302
3,Europe,High,2.854167
4,Europe,Low,2.652632
5,North America,High,2.867299
6,North America,Low,2.882353
7,Oceania,High,3.000000
8,Oceania,Low,3.000000
9,South America,High,2.714286


Ada kalanya kita ingin menampilkan beberapa agregat yang berbeda sekaligus, misal ingin menampilkan nilai minimum, nilai maksimum, dan reratanya sekaligus. Maka kita bisa menggunakan metode `agg()` sebagai fungsi agregat dan memasukkan agregatnya dalam list.

In [31]:
df.groupby('Continent')[['Valuation Number (in B)', 'Years_To_Unicorn']].agg(['min', 'max', 'mean'])

Valuation Number (in B)                Years_To_Unicorn      \
                                  min  max      mean              min max   
Continent                                                                   
Africa                              1    2  1.666667                3  17   
Asia                                1  180  3.528814                0  37   
Europe                              1   46  3.517483                0  98   
North America                       1  100  3.393162                0  28   
Oceania                             1   40  7.000000                4  17   
South America                       1    5  2.285714                2  20   

                         
                   mean  
Continent                
Africa         7.666667  
Asia           6.657627  
Europe         8.251748  
North America  6.897436  
Oceania        7.875000  
South America  7.047619

Atau dengan mengatur spesifikasi berdasarkan kolom dan fungsi agregatnya secara langsung:

In [48]:
df2 = df.groupby('Continent').agg({
    'Valuation Number (in B)': 'mean',
    'Years_To_Unicorn': ['min','max']
})

df2

Valuation Number (in B) Years_To_Unicorn    
                                 mean              min max
Continent                                                 
Africa                       1.666667                3  17
Asia                         3.528814                0  37
Europe                       3.517483                0  98
North America                3.393162                0  28
Oceania                      7.000000                4  17
South America                2.285714                2  20

Indeks hierarki dalam kolom akan terlihat seperti ini:

In [49]:
df2.columns

MultiIndex([('Valuation Number (in B)', 'mean'),
            (       'Years_To_Unicorn',  'min'),
            (       'Years_To_Unicorn',  'max')],
           )

Dengan menggunakan pemahaman list, kita dapat menggabungkan level-level (dalam sebuah tuple) dengan tanda _ pada setiap perulangan.

In [50]:
df2.columns = ['_'.join(agg_col) for agg_col in df2.columns]
df2

,Valuation Number (in B)_mean,Years_To_Unicorn_min,Years_To_Unicorn_max
Continent,,,
Africa,1.666667,3,17
Asia,3.528814,0,37
Europe,3.517483,0,98
North America,3.393162,0,28
Oceania,7.000000,4,17
South America,2.285714,2,20


## **2. Multi Index**

Kita melihat bahwa indeks hierarki dalam kolom df2 di atas sebelumnya berbentuk  Multi Index atau indeks bertingkat. Kita juga bisa mendapatkan Multi Index pada bagian indeks dengan contoh sebagai berikut.


In [52]:
df3 = df.groupby(['Valuation Class', 'Continent'])[['Valuation Number (in B)', 'Years_To_Unicorn']].mean()
df3

Valuation Number (in B)  Years_To_Unicorn
Valuation Class Continent                                               
High            Asia                          9.575000          6.775000
                Europe                        7.750000          9.229167
                North America                 6.976303          6.739336
                Oceania                      16.333333          4.666667
                South America                 4.142857          5.714286
Low             Africa                        1.666667          7.666667
                Asia                          1.279070          6.613953
                Europe                        1.378947          7.757895
                North America                 1.371658          6.986631
                Oceania                       1.400000          9.800000
                South America                 1.357143          7.714286

Kita bisa melihat nama indeks dari Multi Index tersebut dengan menggunakan atribut `names` pada index dari dataframe df3.

In [54]:
df3.index.names

FrozenList(['Valuation Class', 'Continent'])

Kita bisa melakukan indexing baris untuk mengambil index yang **High**.

In [57]:
df3.loc['High']

,Valuation Number (in B),Years_To_Unicorn
Continent,,
Asia,9.575000,6.775000
Europe,7.750000,9.229167
North America,6.976303,6.739336
Oceania,16.333333,4.666667
South America,4.142857,5.714286


Kita juga bisa melakukan indexing baris untuk mengambil index yang **Low** pada `Continent` North America dan Oceania.

In [62]:
df3.loc['Low'].loc[['North America', 'Oceania']]

,Valuation Number (in B),Years_To_Unicorn
Continent,,
North America,1.371658,6.986631
Oceania,1.400000,9.800000


Selanjutnya kita bisa mengambil index yang **Low** pada `Continent` Europe dengan meggunakan syntax `dataframe.loc[index_outside, index_inside]`.

In [71]:
df3.loc[[('Low', 'Europe')]]

,,Valuation Number (in B),Years_To_Unicorn
Valuation Class,Continent,,
Low,Europe,1.378947,7.757895


Selain itu, kita bisa menggunakan metode `xs()` untuk mengembalikan cross-section dari suatu Series/DataFrame. Misal, kita ingin mengambil seluruh data untuk index `Valuation Class` yang **High**.

In [77]:
df3.xs(level='Valuation Class', key='High')

,Valuation Number (in B),Years_To_Unicorn
Continent,,
Asia,9.575000,6.775000
Europe,7.750000,9.229167
North America,6.976303,6.739336
Oceania,16.333333,4.666667
South America,4.142857,5.714286


Atau bisa juga mengambil seluruh data untuk index `Continent` yang **Asia**.

In [80]:
df3

Valuation Number (in B)  Years_To_Unicorn
Valuation Class Continent                                               
High            Asia                          9.575000          6.775000
                Europe                        7.750000          9.229167
                North America                 6.976303          6.739336
                Oceania                      16.333333          4.666667
                South America                 4.142857          5.714286
Low             Africa                        1.666667          7.666667
                Asia                          1.279070          6.613953
                Europe                        1.378947          7.757895
                North America                 1.371658          6.986631
                Oceania                       1.400000          9.800000
                South America                 1.357143          7.714286

In [78]:
df3.xs(level='Continent', key='Asia')

,Valuation Number (in B),Years_To_Unicorn
Valuation Class,,
High,9.57500,6.775000
Low,1.27907,6.613953


In [84]:
pd.__version__

'1.4.3'

In [87]:
df3.xs(level=['Valuation Class', 'Continent'], key=('High','Asia'))

,,Valuation Number (in B),Years_To_Unicorn
Valuation Class,Continent,,
High,Asia,9.575,6.775


## **3. Reshaping and Joining Data**

Terdapat beberapa metode untuk mengubah bentuk dan menggabungkan dataframe. Beberapa metode yang bisa dipakai diantaranya sebagai berikut.

| Method | Description | 
| --- | --- | 
| `melt()` | Unpivot a DataFrame from wide to long format, optionally leaving identifiers set. | 
| `pivot()` | Return reshaped DataFrame organized by given index / column values. |
| `merge()` | Merge DataFrame or named Series objects with a database-style join. |
| `join()` | Join columns of another DataFrame. |
| `pd.concat()` | Concatenate pandas objects along a particular axis. |

Kita akan mempraktikan lebih lanjut terkait kegunaan `melt(), pivot()` dan `merge()` dengan menggunakan dataset `tsa_passenger_throughput.csv`. Sekarang mari kita lihat data jumlah penumpang TSA, yang membandingkan jumlah penumpang tahun 2021 dengan hari yang sama pada tahun 2020 dan 2019.

In [101]:
# parse dates untuk menjadikan kolom 'Date' sebagai datetime
df = pd.read_csv('tsa_passenger_throughput.csv', parse_dates=['Date'])
df

,Date,2021 Traveler Throughput,2020 Traveler Throughput,2019 Traveler Throughput
0,2021-05-14,1716561.0,250467,2664549
1,2021-05-13,1743515.0,234928,2611324
2,2021-05-12,1424664.0,176667,2343675
3,2021-05-11,1315493.0,163205,2191387
4,2021-05-10,1657722.0,215645,2512315
...,...,...,...,...
360,2020-05-19,NaN,190477,2312727
361,2020-05-18,NaN,244176,2615691
362,2020-05-17,NaN,253807,2620276
363,2020-05-16,NaN,193340,2091116


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      365 non-null    datetime64[ns]
 1   2021 Traveler Throughput  134 non-null    float64       
 2   2020 Traveler Throughput  365 non-null    int64         
 3   2019 Traveler Throughput  365 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 11.5 KB


Pertama, kita akan menggunakan huruf kecil pada nama kolom dan mengambil kata pertama (misalnya, 2021 untuk Jumlah Wisatawan Tahun 2021) agar lebih mudah digunakan.

In [105]:
df.columns

Index(['date', '2021', '2020', '2019'], dtype='object')

In [102]:
# cara 1
# df.columns = [i.lower().split()[0] for i in df.columns]
# df

In [104]:
# cara 2
df = df.rename(columns=lambda i: i.lower().split()[0])
df

,date,2021,2020,2019
0,2021-05-14,1716561.0,250467,2664549
1,2021-05-13,1743515.0,234928,2611324
2,2021-05-12,1424664.0,176667,2343675
3,2021-05-11,1315493.0,163205,2191387
4,2021-05-10,1657722.0,215645,2512315
...,...,...,...,...
360,2020-05-19,NaN,190477,2312727
361,2020-05-18,NaN,244176,2615691
362,2020-05-17,NaN,253807,2620276
363,2020-05-16,NaN,193340,2091116


### `Melting`

Melting membantu mengubah data kita menjadi format panjang. Sekarang, kita memiliki semua angka keluaran wisatawan dalam satu kolom.

In [107]:
df_melted = df.melt(
    id_vars='date',                     # kolom yang dipertahankan: date, jadi 2021, 2020, dan 2019 akan dijadikan satu kolom
    var_name='year',                
    value_name='travelers'              # nama kolom value
)

df_melted.sample(5, random_state=1)

,date,year,travelers
974,2020-09-12,2019,1879822.0
435,2021-03-05,2020,2198517.0
1029,2020-07-19,2019,2727355.0
680,2020-07-03,2020,718988.0
867,2020-12-28,2019,2500396.0


Untuk mengubahnya menjadi deret waktu jumlah wisatawan, kita perlu mengganti tahun pada kolom date dengan tahun pada kolom year. Jika tidak, kita akan menandai angka tahun-tahun sebelumnya dengan tahun yang salah.

In [116]:
# mengambil year dari kolom year dan mengambil bulan serta hari dari kolom date
df_melted['date'] = pd.to_datetime(df_melted['year'] + df_melted['date'].dt.strftime('-%m-%d'))
df_melted.sample(5, random_state=1)

,date,year,travelers
974,2019-09-12,2019,1879822.0
435,2020-03-05,2020,2198517.0
1029,2019-07-19,2019,2727355.0
680,2020-07-03,2020,718988.0
867,2019-12-28,2019,2500396.0


Buang nilai nan dengan metode dropna():

In [120]:
df_melted.dropna(inplace=True)

### `Pivoting`

Dengan menggunakan data hasil melting, kita dapat memutar data untuk membandingkan jumlah penumpang TSA pada tanggal 1-10 bulan Maret selama beberapa tahun:

In [129]:
# kondisi 1 - bulan maret
#(df_melted['date'].dt.month == 3)

# kondisi 2 - tanggal 1-10
#(df_melted['date'].dt.day <= 10)

# membuat dataframe yang berisi data tanggal 1-10 bulan maret
df_march = df_melted[(df_melted['date'].dt.month == 3) & (df_melted['date'].dt.day <= 10)].copy()
df_march 

# menambah kolom 'days_in_march'
df_march['days_in_march'] = df_march['date'].dt.day
df_march

# membuat tabel pivot
df_pivot = df_march.pivot(
    index='year',
    columns='days_in_march',
    values='travelers'
)

df_pivot

days_in_march,1,2,3,4,5,6,7,8,9,10
year,,,,,,,,,,
2019,2257920.0,1979558.0,2143619.0,2402692.0,2543689.0,2156262.0,2485430.0,2378673.0,2122898.0,2187298.0
2020,2089641.0,1736393.0,1877401.0,2130015.0,2198517.0,1844811.0,2119867.0,1909363.0,1617220.0,1702686.0
2021,1049692.0,744812.0,826924.0,1107534.0,1168734.0,992406.0,1278557.0,1119303.0,825745.0,974221.0


Atribut T menyediakan cara cepat untuk membalik baris dan kolom:

In [130]:
df_pivot.T

year,2019,2020,2021
days_in_march,,,
1,2257920.0,2089641.0,1049692.0
2,1979558.0,1736393.0,744812.0
3,2143619.0,1877401.0,826924.0
4,2402692.0,2130015.0,1107534.0
5,2543689.0,2198517.0,1168734.0
6,2156262.0,1844811.0,992406.0
7,2485430.0,2119867.0,1278557.0
8,2378673.0,1909363.0,1119303.0
9,2122898.0,1617220.0,825745.0


### `Merging`

Kita biasanya mengamati perubahan dalam perjalan udara di sekitar hari libur, jadi menambahkan informasi tentang tanggal di dataset TSA akan memberikan lebih banyak konteks. File `holidays.csv` berisi beberapa hari libur besar di Amerika Serikat.

In [133]:
holidays = pd.read_csv('holidays.csv', parse_dates=['date'])
holidays

,date,holiday
0,2019-01-01,New Year's Day
1,2019-05-27,Memorial Day
2,2019-07-04,July 4th
3,2019-09-02,Labor Day
4,2019-11-28,Thanksgiving
5,2019-12-24,Christmas Eve
6,2019-12-25,Christmas Day
7,2019-12-31,New Year's Eve
8,2020-01-01,New Year's Day
9,2020-05-25,Memorial Day


Menggabungkan hari libur dengan data jumlah wisatawan TSA akan memberikan lebih banyak konteks untuk analisis kita:

In [138]:
df_melted_holiday = df_melted.merge(holidays, on='date', how='left').sort_values('date')
df_melted_holiday.head()

,date,year,travelers,holiday
632,2019-01-01,2019,2126398.0,New Year's Day
631,2019-01-02,2019,2345103.0,NaN
630,2019-01-03,2019,2202111.0,NaN
629,2019-01-04,2019,2150571.0,NaN
628,2019-01-05,2019,1975947.0,NaN


Kita dapat mengambil langkah lebih jauh dengan menandai dua (2) hari sebelum dan satu (1) hari sesudah setiap hari libur sebagai bagian dari hari libur. Hal ini akan memudahkan untuk membandingkan perjalanan liburan dari tahun ke tahun dan mencari peningkatan perjalanan di sekitar hari libur:

In [141]:
df_melted_holiday_travel = df_melted_holiday.copy()
df_melted_holiday_travel['holiday'] = df_melted_holiday_travel['holiday'].fillna(method='ffill', limit=1).fillna(method='bfill', limit=2)
df_melted_holiday_travel.head()

,date,year,travelers,holiday
632,2019-01-01,2019,2126398.0,New Year's Day
631,2019-01-02,2019,2345103.0,New Year's Day
630,2019-01-03,2019,2202111.0,NaN
629,2019-01-04,2019,2150571.0,NaN
628,2019-01-05,2019,1975947.0,NaN


In [142]:
df_melted_holiday_travel[df_melted_holiday_travel['holiday'].notnull()]

,date,year,travelers,holiday
632,2019-01-01,2019,2126398.0,New Year's Day
631,2019-01-02,2019,2345103.0,New Year's Day
853,2019-05-25,2019,2512237.0,Memorial Day
852,2019-05-26,2019,2453649.0,Memorial Day
851,2019-05-27,2019,2269035.0,Memorial Day
850,2019-05-28,2019,2485770.0,Memorial Day
815,2019-07-02,2019,2088760.0,July 4th
814,2019-07-03,2019,2184253.0,July 4th
813,2019-07-04,2019,2345846.0,July 4th
812,2019-07-05,2019,2795369.0,July 4th


Dataframe tersebut nantinya akan kita olah lebih lanjut pada bagian berikutnya. Namun untuk memperdalam pemahaman terkait merge, join, dan concat kita akan membahasnya terlebih dulu secara detail.

### `merge()` vs `join()` vs `pd.concat()`

| Method | Description | Default |
| --- | --- | --- |
| `merge()` | Menggabungkan berdasarkan kolom | Inner Join |
| `join()` | Menggabungkan berdasarkan indeks | Left Join |
| `concat()` | Menempelkan berdasarkan axis | Outer Join |

#### **merge**

In [143]:
df_kiri = pd.DataFrame({
    'Key' : ['A', 'B', 'C', 'D'],
    'Jakarta' : [1, 2, 3, 4],
    'Bogor' : [5, 6, 7, 8]
})

df_kanan = pd.DataFrame({
    'Key' : ['A', 'B', 'C', 'E'],
    'Surabaya' : [10, 20, 30, 40],
    'Bandung' : [50, 60, 70, 80]
})

# menampilkan lebih dari 1 dataframe sekaligus
display(df_kiri, df_kanan)

,Key,Jakarta,Bogor
0,A,1,5
1,B,2,6
2,C,3,7
3,D,4,8


,Key,Surabaya,Bandung
0,A,10,50
1,B,20,60
2,C,30,70
3,E,40,80


Secara default, `merge()` hanya akan menampilkan baris dengan nilai pada kolom **Key** yang sama. Urutan kolom dari dataframe pada parameter left diikuti kolom dari dataframa pada parameter right.

In [144]:
pd.merge(left=df_kiri, right=df_kanan, on='Key')

,Key,Jakarta,Bogor,Surabaya,Bandung
0,A,1,5,10,50
1,B,2,6,20,60
2,C,3,7,30,70


Untuk mengatur cara join, kita bisa menambahkan parameter `how` dan memasukkan argumen `left, right, inner, outer,` atau `cross`. Kita juga bisa menambahkan `indicator=True` untuk mengetahui **key**-nya ada pada tabel yang mana.

In [146]:
# inner join
pd.merge(left=df_kiri, right=df_kanan, how='inner', on='Key', indicator=True)

,Key,Jakarta,Bogor,Surabaya,Bandung,_merge
0,A,1,5,10,50,both
1,B,2,6,20,60,both
2,C,3,7,30,70,both


In [147]:
# left join
pd.merge(left=df_kiri, right=df_kanan, how='left', on='Key', indicator=True)

,Key,Jakarta,Bogor,Surabaya,Bandung,_merge
0,A,1,5,10.0,50.0,both
1,B,2,6,20.0,60.0,both
2,C,3,7,30.0,70.0,both
3,D,4,8,NaN,NaN,left_only


In [154]:
# right join
pd.merge(left=df_kiri, right=df_kanan, how='right', on='Key', indicator=True)

,Key,Jakarta,Bogor,Surabaya,Bandung,_merge
0,A,1.0,5.0,10,50,both
1,B,2.0,6.0,20,60,both
2,C,3.0,7.0,30,70,both
3,E,NaN,NaN,40,80,right_only


In [150]:
# outer join
pd.merge(left=df_kiri, right=df_kanan, how='outer', on='Key', indicator=True)

,Key,Jakarta,Bogor,Surabaya,Bandung,_merge
0,A,1.0,5.0,10.0,50.0,both
1,B,2.0,6.0,20.0,60.0,both
2,C,3.0,7.0,30.0,70.0,both
3,D,4.0,8.0,NaN,NaN,left_only
4,E,NaN,NaN,40.0,80.0,right_only


In [152]:
# cross join
pd.merge(left=df_kiri, right=df_kanan, how='cross', indicator=True)

,Key_x,Jakarta,Bogor,Key_y,Surabaya,Bandung,_merge
0,A,1,5,A,10,50,both
1,A,1,5,B,20,60,both
2,A,1,5,C,30,70,both
3,A,1,5,E,40,80,both
4,B,2,6,A,10,50,both
5,B,2,6,B,20,60,both
6,B,2,6,C,30,70,both
7,B,2,6,E,40,80,both
8,C,3,7,A,10,50,both
9,C,3,7,B,20,60,both


Kita juga bisa menggabungkan 2 DataFrame menggunakan kolom yang namanya berbeda.

In [155]:
df_kiri = df_kiri.rename(columns={'Key':'Id'})
display(df_kiri, df_kanan)

,Id,Jakarta,Bogor
0,A,1,5
1,B,2,6
2,C,3,7
3,D,4,8


,Key,Surabaya,Bandung
0,A,10,50
1,B,20,60
2,C,30,70
3,E,40,80


In [156]:
pd.merge(df_kiri, df_kanan, how='left', left_on='Id', right_on='Key')

,Id,Jakarta,Bogor,Key,Surabaya,Bandung
0,A,1,5,A,10.0,50.0
1,B,2,6,B,20.0,60.0
2,C,3,7,C,30.0,70.0
3,D,4,8,NaN,NaN,NaN


In [158]:
# cara penulisan lain
df_test = df_kiri.merge(df_kanan, how='left', left_on='Id', right_on='Key')

In [161]:
df_test.loc[0, 'Bandung'] = np.nan
df_test

,Id,Jakarta,Bogor,Key,Surabaya,Bandung
0,A,1,5,A,10.0,NaN
1,B,2,6,B,20.0,60.0
2,C,3,7,C,30.0,70.0
3,D,4,8,NaN,NaN,NaN


Jika ada nama kolom yang sama, maka akan diberi akhiran _x untuk tabel kiri dan akhiran _y untuk tabel kanan

In [165]:
df_kiri_new = df_kiri.reset_index()
df_kanan_new = df_kanan.reset_index()

display(df_kiri_new, df_kanan_new)

,index,Id,Jakarta,Bogor
0,0,A,1,5
1,1,B,2,6
2,2,C,3,7
3,3,D,4,8


,index,Key,Surabaya,Bandung
0,0,A,10,50
1,1,B,20,60
2,2,C,30,70
3,3,E,40,80


In [166]:
# kolom yang sama akan diberi akhiran _x untuk yang kiri dan _y untuk yang kanan
df_kiri_new.merge(df_kanan_new, how='left', left_on='Id', right_on='Key')

,index_x,Id,Jakarta,Bogor,index_y,Key,Surabaya,Bandung
0,0,A,1,5,0.0,A,10.0,50.0
1,1,B,2,6,1.0,B,20.0,60.0
2,2,C,3,7,2.0,C,30.0,70.0
3,3,D,4,8,NaN,NaN,NaN,NaN


Kita juga dapat menggabungkan dataframe menggunakan lebih dari 1 kolom.

In [167]:
df_kiri_new = df_kiri_new.rename(columns={'Id':'Key'})
display(df_kiri_new, df_kanan_new)

,index,Key,Jakarta,Bogor
0,0,A,1,5
1,1,B,2,6
2,2,C,3,7
3,3,D,4,8


,index,Key,Surabaya,Bandung
0,0,A,10,50
1,1,B,20,60
2,2,C,30,70
3,3,E,40,80


In [168]:
df_kiri_new.merge(df_kanan_new, on=['index', 'Key'])

,index,Key,Jakarta,Bogor,Surabaya,Bandung
0,0,A,1,5,10,50
1,1,B,2,6,20,60
2,2,C,3,7,30,70


#### **join**

In [169]:
df_kiri = pd.DataFrame({
    'Jakarta' : [1, 2, 3, 4],
    'Bogor' : [5, 6, 7, 8]
}, index=['A', 'B', 'C', 'D'])

df_kanan = pd.DataFrame({
    'Surabaya' : [10, 20, 30, 40],
    'Bandung' : [50, 60, 70, 80]
}, index=['A', 'B', 'C', 'E'])

display(df_kiri, df_kanan)

,Jakarta,Bogor
A,1,5
B,2,6
C,3,7
D,4,8


,Surabaya,Bandung
A,10,50
B,20,60
C,30,70
E,40,80


Terdapat 2 cara penulisan untuk melakukan join.

In [172]:
# cara 1
pd.DataFrame.join(df_kiri, df_kanan)

# by default: left join. jadi yang ditampilkan seluruh indeks yang disebelah kiri

,Jakarta,Bogor,Surabaya,Bandung
A,1,5,10.0,50.0
B,2,6,20.0,60.0
C,3,7,30.0,70.0
D,4,8,NaN,NaN


In [173]:
df_kiri.join(df_kanan)

,Jakarta,Bogor,Surabaya,Bandung
A,1,5,10.0,50.0
B,2,6,20.0,60.0
C,3,7,30.0,70.0
D,4,8,NaN,NaN


Terlihat bahwa dataframe yang disebutkan pertama (df_kiri) berperan sebagai tabel kiri dan dataframe yang disebutkan kedua (df_kanan) berperan sebagai tabel kanan. Karena df_kiri tidak memiliki index E, sedangkan df_kanan tidak memiliki index D, maka nilai yang ditampilkan pada Surabaya dan Bandung adalah NaN pada index ke D.

Secara default `join()` akan menggabungkan secara left join. Untuk mengatur cara join, kita bisa menambahkan parameter `how`.

In [174]:
# inner
df_kiri.join(df_kanan, how='inner')

,Jakarta,Bogor,Surabaya,Bandung
A,1,5,10,50
B,2,6,20,60
C,3,7,30,70


In [175]:
# right
df_kiri.join(df_kanan, how='right')

,Jakarta,Bogor,Surabaya,Bandung
A,1.0,5.0,10,50
B,2.0,6.0,20,60
C,3.0,7.0,30,70
E,NaN,NaN,40,80


In [176]:
# outer
df_kiri.join(df_kanan, how='outer')

,Jakarta,Bogor,Surabaya,Bandung
A,1.0,5.0,10.0,50.0
B,2.0,6.0,20.0,60.0
C,3.0,7.0,30.0,70.0
D,4.0,8.0,NaN,NaN
E,NaN,NaN,40.0,80.0


Fungsi `join()` tidak bisa menggabungkan dua tabel yang memiliki nama kolom yang sama hanya dengan menggunakan parameter `how`.

In [178]:
df_kiri_new = df_kiri.reset_index()
df_kanan_new = df_kanan.reset_index()

display(df_kiri_new, df_kanan_new)

,index,Jakarta,Bogor
0,A,1,5
1,B,2,6
2,C,3,7
3,D,4,8


,index,Surabaya,Bandung
0,A,10,50
1,B,20,60
2,C,30,70
3,E,40,80


In [181]:
# df_kiri_new.join(df_kanan_new, how='outer')

Jika ada yang nama kolomnya sama maka harus ditambahkan parameter lsuffix dan rsuffix untuk menambahkan akhiran pada nama kolom tersebut.

In [182]:
df_kiri_new.join(df_kanan_new, how='outer', lsuffix='_kiri', rsuffix='_kanan')

,index_kiri,Jakarta,Bogor,index_kanan,Surabaya,Bandung
0,A,1,5,A,10,50
1,B,2,6,B,20,60
2,C,3,7,C,30,70
3,D,4,8,E,40,80


#### **concatenate**

In [183]:
df_A = pd.DataFrame({
    'Jakarta' : [1, 2, 3, 4],
    'Bogor' : [5, 6, 7, 8],
    'Makassar' : [9, 10, 11, 12]
}, index=[0, 1, 2, 3])

df_B = pd.DataFrame({
    'Jakarta' : [10, 20, 30, 40],
    'Bogor' : [50, 60, 70, 80],
    'Makassar' : [90, 100, 110, 120]
}, index=[4, 5, 6, 7])

df_C = pd.DataFrame({
    'Bali' : [100, 200, 300, 400],
    'Papua' : [500, 600, 700, 800],
    'Riau' : [900, 1000, 1100, 1200]
}, index=[0, 1, 2, 3])

display(df_A, df_B, df_C)

,Jakarta,Bogor,Makassar
0,1,5,9
1,2,6,10
2,3,7,11
3,4,8,12


,Jakarta,Bogor,Makassar
4,10,50,90
5,20,60,100
6,30,70,110
7,40,80,120


,Bali,Papua,Riau
0,100,500,900
1,200,600,1000
2,300,700,1100
3,400,800,1200


Secara default, `pd.concat()` akan menambahkan baris pada axis=0 (menempelkan ke bawah)

In [184]:
# dataframe yang mau digabung harus dimasukkan dalam list
pd.concat([df_A, df_B])

,Jakarta,Bogor,Makassar
0,1,5,9
1,2,6,10
2,3,7,11
3,4,8,12
4,10,50,90
5,20,60,100
6,30,70,110
7,40,80,120


In [185]:
pd.concat([df_A, df_C])

,Jakarta,Bogor,Makassar,Bali,Papua,Riau
0,1.0,5.0,9.0,NaN,NaN,NaN
1,2.0,6.0,10.0,NaN,NaN,NaN
2,3.0,7.0,11.0,NaN,NaN,NaN
3,4.0,8.0,12.0,NaN,NaN,NaN
0,NaN,NaN,NaN,100.0,500.0,900.0
1,NaN,NaN,NaN,200.0,600.0,1000.0
2,NaN,NaN,NaN,300.0,700.0,1100.0
3,NaN,NaN,NaN,400.0,800.0,1200.0


Terlihat bahwa dataframe dengan nama kolom yang sama akan ditempelkan pada satu kolom yang sama.

Untuk menempelkan ke samping atau menambahkan pada kolom, kita bisa menambahkan parameter `axis=1`.

In [186]:
pd.concat([df_A, df_B], axis=1)

,Jakarta,Bogor,Makassar,Jakarta,Bogor,Makassar
0,1.0,5.0,9.0,NaN,NaN,NaN
1,2.0,6.0,10.0,NaN,NaN,NaN
2,3.0,7.0,11.0,NaN,NaN,NaN
3,4.0,8.0,12.0,NaN,NaN,NaN
4,NaN,NaN,NaN,10.0,50.0,90.0
5,NaN,NaN,NaN,20.0,60.0,100.0
6,NaN,NaN,NaN,30.0,70.0,110.0
7,NaN,NaN,NaN,40.0,80.0,120.0


In [187]:
pd.concat([df_A, df_C], axis=1)

,Jakarta,Bogor,Makassar,Bali,Papua,Riau
0,1,5,9,100,500,900
1,2,6,10,200,600,1000
2,3,7,11,300,700,1100
3,4,8,12,400,800,1200


Terlihat bahwa dataframe dengan nama indeks yang sama akan ditempelkan pada indeks yang sama.

Fungsi `pd.concat()` akan menampilkan data sesuai urutan yang disebutkan.

In [188]:
pd.concat([df_A, df_C, df_B])

,Jakarta,Bogor,Makassar,Bali,Papua,Riau
0,1.0,5.0,9.0,NaN,NaN,NaN
1,2.0,6.0,10.0,NaN,NaN,NaN
2,3.0,7.0,11.0,NaN,NaN,NaN
3,4.0,8.0,12.0,NaN,NaN,NaN
0,NaN,NaN,NaN,100.0,500.0,900.0
1,NaN,NaN,NaN,200.0,600.0,1000.0
2,NaN,NaN,NaN,300.0,700.0,1100.0
3,NaN,NaN,NaN,400.0,800.0,1200.0
4,10.0,50.0,90.0,NaN,NaN,NaN
5,20.0,60.0,100.0,NaN,NaN,NaN


## **4. Aggregations and Grouping using pivot**

Kita akan melanjukan study case sebelumnya. Kita bisa membuat tabel pivot untuk membandingkan perjalanan liburan dari tahun ke tahun dalam kumpulan data kita:

In [190]:
df_melted_holiday_travel.pivot_table(
    index='year',                       # nilai unik dari kolom yang akan dijadikan indeks
    columns='holiday',                  # nilai unik dari kolom yang akan dijadikan kolom
    values='travelers',                 # nilai yang akan diagregasi
    aggfunc='sum'                       # agregat function, by default 'mean'
)

holiday,Christmas Day,Christmas Eve,July 4th,Labor Day,Memorial Day,New Year's Day,New Year's Eve,Thanksgiving
year,,,,,,,,
2019,5053366.0,6470862.0,9414228.0,8314811.0,9720691.0,4471501.0,6535464.0,9090478.0
2020,1745242.0,3029810.0,2682541.0,2993653.0,1126253.0,4490388.0,3057449.0,3364358.0
2021,NaN,NaN,NaN,NaN,NaN,1998871.0,NaN,NaN


Selanjutnya, kita mengelompokkan Malam Natal dan Hari Natal, juga untuk Malam Tahun Baru dan Hari Tahun baru, dan membuat tabel pivot:

In [209]:
df_melted_holiday_travel

,date,year,travelers,holiday
0,2019-01-01,2019,2126398.0,New Year's Day
1,2019-01-02,2019,2345103.0,New Year's Day
2,2019-01-03,2019,2202111.0,NaN
3,2019-01-04,2019,2150571.0,NaN
4,2019-01-05,2019,1975947.0,NaN
...,...,...,...,...
859,2021-05-10,2021,1657722.0,NaN
860,2021-05-11,2021,1315493.0,NaN
861,2021-05-12,2021,1424664.0,NaN
862,2021-05-13,2021,1743515.0,NaN


In [228]:
df_holiday = df_melted_holiday_travel.assign(
    holiday=lambda x: np.where(
        x.holiday.str.contains('New|Christmas', regex=True),
        x.holiday.str.replace('Eve|Day', '', regex=True),
        x.holiday)
)

df_holiday.pivot_table(
    index='year',                       # nilai unik dari kolom yang akan dijadikan indeks
    columns='holiday',                  # nilai unik dari kolom yang akan dijadikan kolom
    values='travelers',                 # nilai yang akan diagregasi
    aggfunc='sum',                       # agregat function, by default 'mean'
    margins=True,                       # menambahkan total pada 
    margins_name='Total'
)

holiday,Christmas,July 4th,Labor Day,Memorial Day,New Year's,Thanksgiving,Total
year,,,,,,,
2019,11524228.0,9414228.0,8314811.0,9720691.0,11006965.0,9090478.0,59071401.0
2020,4775052.0,2682541.0,2993653.0,1126253.0,7547837.0,3364358.0,22489694.0
2021,NaN,NaN,NaN,NaN,1998871.0,NaN,1998871.0
Total,16299280.0,12096769.0,11308464.0,10846944.0,20553673.0,12454836.0,83559966.0


Dengan menggunakan metode `.agg()` kita dapat menetukan berapa pun jumlahnya:

In [231]:
df_melted_holiday_travel.assign(
    holiday_travelers=lambda x: np.where(x.holiday.notnull(), x.travelers, np.nan),
    non_holiday_travelers=lambda x: np.where(x.holiday.isna(), x.travelers, np.nan)
).groupby('year')[['holiday_travelers', 'non_holiday_travelers']].agg(['mean', 'std'])

holiday_travelers                non_holiday_travelers               
                  mean            std                  mean            std
year                                                                      
2019      2.271977e+06  303021.675751          2.312359e+06  283906.226598
2020      8.649882e+05  489938.240989          8.831619e+05  650399.772930
2021      9.994355e+05  273573.249680          1.114347e+06  339479.298658